In [96]:
# ~ IMPORTS ~
import qrcode
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from pyzbar.pyzbar import decode, ZBarSymbol
import cv2
import oss
import ast

In [354]:
# ~ PARAMETERS ~

# Metadata for QR code

# Example dictionary
# qrMetadata = {
#     "user": "Zoe",
#     "date": "2/19/2020",
#     "experiment": "PI Stuff",
#     "mood": "stressed",
#     "volenum": "1234"
# }

# Create the dictonary
qrMetadata = {
    "user": "Chase",
    "date": "2/19/2020",
    "experiment": "QR Code",
    "coffee": "level low"
}

# Change the color of the QR code
qrColor = "black"

# Change based on wether the QR is for position or vole
# Values: Position, Vole
qrType = "Vole"

# Change based on where the QR will be placed.
# Values: TL, TR, BL, BR - (Top Left, Top Right, Bottom Left, Bottom Right)
location = "BR"

# Change based on the vole number
voleNum = "1234"

# Directory used to save the QR code
fileDir = r'D:\Donaldson Lab\QRCodes'

# Name of the individual QR code
fileName = r'\test'

# Image type (ex. png, bmp, jpeg)
extension = "png"

# ~ DON'T CHANGE ~
# Creates a full filepath using the direcotry and filename
fileLocation = fileDir + fileName + '.' + extension

In [380]:
# ~ QR Encoder ~ 
def QREncoder():
    # Create qr code instance
    qrEncoder = qrcode.QRCode(
        version = 1,
        error_correction = qrcode.constants.ERROR_CORRECT_H,
        box_size = 10,
        border = 4,
    )

    # Add data
    qrEncoder.add_data(qrMetadata)
    qrEncoder.make(fit=True)

    # Create an image from the QR Code instance
    img = qrEncoder.make_image(fill_color= qrColor)
    
    # Create directory if it doesn't exist
    try:
        os.mkdir(fileDir)
    except:
        pass
    
    # Save QR to desired filepath
    img.save(fileLocation)
    
    # Open the QR again, this time in color
    img = Image.open(fileLocation).convert("RGBA")

    # Get the width and height of the created QR
    width, height = img.size
   
    # Draw QR code 
    draw = ImageDraw.Draw(img)

    # If it is a positional QR code, then draw boxes
    if qrType == "Position":
        # Draw red outter box
        draw.rectangle([(40,40),(109,109)], outline="red", width = 10)
        # Draw red inner box
        draw.rectangle([(60,60),(89,89)], fill="red")
    
        # Check which corner the QR is located, rotate point of interest to that location
        if location == "TR":
            img = img.rotate(270)
        elif location == "BL":
            img = img.rotate(90)
        elif location == "BR":
            img = img.rotate(180)
        else:
            pass
    # If it is a vole QR code, then draw text showing the vole number
    elif qrType == "Vole":
        # Declare fonts
        topTextFont = ImageFont.truetype("arial.ttf", 35)
        # Draw text that displays the vole number in black
        draw.text((0, 0), "Vole Number: %s" % voleNum, font=topTextFont, fill="black")
    else:
        pass

    # Save it to desired filepath, change the extension as needed:
    img.save(fileLocation)

In [384]:
# ~ QR READER ~ 
def QRDecoder(frame):
    # Returns QR codes from the frame. Can either be from cv2.imread or Image.open
    # Decode returns a list of objects
    # [ 
    #   data = b'...' => The QR metadata
    #   rect = Rectangle(...) => This is a rectangle object representing the size of the QR
    #   Polygon = [Point(...)...] => A Polygon object to show where the QR was located in the image.
    # ] 
    decoded = decode(frame, symbols=[ZBarSymbol.QRCODE])
    
    # List to store the dictionary values from the QR codes
    dicts = []
    
    # Add the QR dictionaries to the list
    for qr in decoded:
        # Convert B string info to dictionaries
        dict = ast.literal_eval(qr[0].decode('utf-8'))
        
        print(qr[3])
        # Add location of POI to dictionary
        
        
        
        # Append the dictionary to the list
        dicts.append(dict)
        
    # Noticed that decode inserts items from bottom to top and right to left.
    # Reversing the list puts the top left QR code first
    dicts.reverse()
    
    # TESTING: print the qr codes found
    for qr in dicts:
        print(qr)
   
    # RETURN: List of dictonaries from all decoded QR codes.
    return dicts

In [392]:
QREncoder()
qrCodesFound = QRDecoder(cv2.imread(r'D:\Donaldson Lab\QRCodes\Tests\test_TL.png'))

[Point(x=40, y=40), Point(x=42, y=569), Point(x=569, y=569), Point(x=569, y=42)]
{'user': 'Chase', 'date': '2/19/2020', 'experiment': 'QR Code', 'coffee': 'level low'}
